# SECTION 0 — Setup & Imports

## Regression Solver Benchmark

This notebook demonstrates the implementation and performance comparison of various optimization algorithms for solving regularized linear regression problems.

We test:
- ISTA (Iterative Soft-Thresholding Algorithm)
- FISTA (Fast ISTA)
- Gradient Descent (as a baseline)
- (Optionally) L-BFGS for smooth problems

The models handled include:
- LASSO (ℓ₁-regularized least squares)
- Ridge (ℓ₂-regularized least squares)
- Elastic Net (ℓ₁ + ℓ₂ regularization)

---

This notebook relies on external modular implementations found in the `prox/` and `solvers/` directories.

In [ ]:
# --- Imports ---
import numpy as np
import matplotlib.pyplot as plt
import time

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

# --- Prox operators ---
from prox.prox_l1 import prox_l1
from prox.prox_l2 import prox_l2
from prox.prox_elasticnet import prox_elasticnet

# --- Solvers ---
from solvers.ista import ista
from solvers.fista import fista
from solvers.gradient import gradient_descent
from solvers.lbfgs import lbfgs_solver

# SECTION 1 — Dataset Loading & Preprocessing

## 1. Dataset Loading & Preprocessing

We use a real-world dataset for benchmarking the performance of our solvers.
The dataset is expected to be a CSV file located in the `data/` folder.

We perform:
- Feature-target separation
- Standardization of input features
- Train/test split (80/20)

**Note**: Modify the path and column names below according to your specific dataset.

In [ ]:
import pandas as pd
from pathlib import Path

# --- Load dataset (customize this part!) ---
dataset_path = Path("data") / "your_dataset.csv"  # TODO: replace with actual filename
df = pd.read_csv(dataset_path)

# --- Configure target and features ---
target_column = "target"  # TODO: replace with actual target name
feature_columns = [col for col in df.columns if col != target_column]

X = df[feature_columns].values
y = df[target_column].values

# --- Standardize features ---
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# --- Train/test split ---
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# --- Confirm shapes ---
print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")

# SECTION 2 — Solver Runner Utility

## 2. Solver Runner

This section defines the `run_solver` function, which:
- Selects the appropriate solver (ISTA, FISTA, etc.)
- Selects the correct proximity operator based on the regularization model
- Runs the solver and collects performance metrics

All solvers return:
- Final solution `x`
- Objective function history
- Execution time

In [ ]:
def run_solver(
    method: str,
    model: str,
    A: np.ndarray,
    b: np.ndarray,
    lam: float,
    alpha: float = 0.5,
    max_iter: int = 1000,
    tol: float = 1e-6,
    verbose: bool = False
) -> dict:
    """
    method: 'ista', 'fista', 'gradient', 'lbfgs'
    model: 'lasso', 'ridge', 'elasticnet'
    lam: regularization parameter
    alpha: elastic net mixing parameter (0 = ridge, 1 = lasso)
    """
    # Select proximity operator
    if model == "lasso":
        prox_op = prox_l1
    elif model == "ridge":
        prox_op = prox_l2
    elif model == "elasticnet":
        prox_op = lambda x, lam_: prox_elasticnet(x, lam_, alpha)
    else:
        raise ValueError(f"Unknown model '{model}'")

    # Select solver
    start_time = time.time()

    if method == "ista":
        result = ista(A, b, lam, prox_op, max_iter, tol, verbose)
    elif method == "fista":
        result = fista(A, b, lam, prox_op, max_iter, tol, verbose)
    elif method == "gradient":
        result = gradient_descent(A, b, lam, max_iter, tol, verbose)
    elif method == "lbfgs":
        result = lbfgs_solver(A, b, lam, model=model)
    else:
        raise ValueError(f"Unknown method '{method}'")

    result["time"] = time.time() - start_time
    return result

# SECTION 3 — Run Solvers on the Dataset

## 3. Run Solvers

We now run the selected optimization algorithms on the chosen dataset.

You can choose:
- The regularization model (`lasso`, `ridge`, `elasticnet`)
- The optimization method (`ista`, `fista`, `gradient`, `lbfgs`)
- The regularization parameter `lambda`
- (Optional) Elastic Net mixing parameter `alpha`

The results include the final solution, convergence history, runtime, and more.

In [ ]:
# --- Configuration ---
model = "lasso"           # 'lasso', 'ridge', 'elasticnet'
method = "fista"          # 'ista', 'fista', 'gradient', 'lbfgs'
lam = 0.1                 # regularization strength
alpha = 0.5               # only used for elastic net

# --- Run selected solver ---
result = run_solver(
    method=method,
    model=model,
    A=X_train,
    b=y_train,
    lam=lam,
    alpha=alpha,
    max_iter=1000,
    tol=1e-6,
    verbose=True
)

# --- Unpack results ---
x_sol = result["x"]
history = result.get("history", [])
elapsed_time = result["time"]

print(f"Final objective value: {history[-1] if history else 'N/A'}")
print(f"Elapsed time: {elapsed_time:.4f} seconds")

# SECTION 4 — Evaluation Metrics

## 4. Evaluation

Once the solver has produced a solution vector `x`, we evaluate its quality on the test set.

We report:
- Test Mean Squared Error (MSE)
- Training MSE (optional)
- Sparsity of the solution: percentage of near-zero elements in `x`

In [ ]:
def compute_sparsity(x: np.ndarray, threshold: float = 1e-6) -> float:
    """Return the fraction of coefficients in x that are effectively zero."""
    return np.sum(np.abs(x) < threshold) / len(x)

# --- Predict on test set ---
y_pred = X_test @ x_sol
test_mse = mean_squared_error(y_test, y_pred)
train_mse = mean_squared_error(y_train, X_train @ x_sol)
sparsity = compute_sparsity(x_sol)

# --- Print metrics ---
print(f"Train MSE: {train_mse:.4f}")
print(f"Test MSE:  {test_mse:.4f}")
print(f"Sparsity:  {sparsity * 100:.2f}% zeros in solution vector")

# SECTION 5 — Convergence and Sparsity Plots

## 5. Convergence and Sparsity Plots

We visualize:
- The convergence of the objective value over iterations
- The sparsity of the solution (optional: per iteration if tracked)

This helps assess:
- The speed of convergence
- The stability and smoothness of the solver trajectory
- How aggressively each method induces sparsity (for LASSO/Elastic Net)

In [ ]:
# --- Plot objective function history ---
if history:
    plt.figure(figsize=(8, 4))
    plt.plot(history, label=f"{method.upper()} - {model}")
    plt.xlabel("Iteration")
    plt.ylabel("Objective Value")
    plt.title("Convergence Curve")
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()
else:
    print("No convergence history available to plot.")

### Optional Extension for Later
If you later track sparsity at each iteration inside your solver (e.g., by appending to result["sparsity_history"]), you could add:

In [ ]:
sparsity_history = result.get("sparsity_history", None)
if sparsity_history:
    plt.plot(sparsity_history)
    ...

# SECTION 6 — Summary & Multi-Solver Comparison

## 6. Summary and Comparison

To compare multiple solvers or models, we run several configurations and collect:

- Final objective value
- Runtime (in seconds)
- Test MSE
- Solution sparsity

We display these results in a summary table for easy analysis.

### Example Benchmark Loop

In [ ]:
methods = ["ista", "fista", "gradient", "lbfgs"]
model = "lasso"  # or 'ridge', 'elasticnet'
lam = 0.1
alpha = 0.5

benchmark_results = []

for method in methods:
    print(f"\nRunning: {method.upper()}...")
    result = run_solver(
        method=method,
        model=model,
        A=X_train,
        b=y_train,
        lam=lam,
        alpha=alpha,
        max_iter=1000,
        tol=1e-6,
        verbose=False
    )
    
    x_sol = result["x"]
    y_pred = X_test @ x_sol
    test_mse = mean_squared_error(y_test, y_pred)
    sparsity = compute_sparsity(x_sol)
    time_taken = result["time"]
    final_obj = result.get("history", [None])[-1]

    benchmark_results.append({
        "method": method,
        "final_obj": final_obj,
        "test_mse": test_mse,
        "sparsity": sparsity,
        "time (s)": time_taken
    })

### Display Results Table

In [ ]:
# --- Show results as a DataFrame ---
import pandas as pd

df_results = pd.DataFrame(benchmark_results)
df_results = df_results.sort_values(by="test_mse")
display(df_results)